In [ ]:
# Import y Configuracion

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras import layers, models, callbacks
from sklearn.pipeline import Pipeline

# Reproducibilidad: Sirve para evitar datos duplicados. De esta manera, no sale otro dato si vuelvo a correr el codigo.
# Inicializacion aleatoria de pesos
# Seleccion de lotes (batches) por epach
# Division de Train - Test
# Calculo de GPU (no determinista)
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow: ", tf.__version__)

# 1. Cargar el Dataset Titanic

# Fuente Abierta
URL = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(URL)

print("Shape bruto: ", df.shape)
print("Cols: ", list(df.columns))

# 2. Seleccion de variables y objetivo (y)
# Objetivo: Survived (0/1)
# Cuando la columna es muy obvia, o no dice nada o nada para predecir, puede haber una fuga o error. Si tenemos varias columnas, no usamos todas necesariamente.
y = df["Survived"].astype(int).values

#Tomamos columnas usuales y evitamos leakage (PassengerId, name, Ticket, CABIN NO ayudan de base)

X = df[[
    "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"
]].copy()

# 3. Feature engineering liviano y limpio

# 3.1 Imputacion simple (dejar NaN y luego imputamos con SimpleImputer)
# 3.2 Crear FamilySize y IsAlone (suma de SibSp+Parch y binario)

X["FamilySize"] = X["SibSp"].fillna(0) + X["Parch"].fillna(0) + 1
X["IsAlone"] = (X["FamilySize"] == 1).astype(int)

# 4. Definicion de columnas por tipo
num_cols = ["Age", "SibSp", "Parch", "Fare", "FamilySize"]
cat_cols = ["Pclass", "Sex", "Embarked"] # Pclass como categorica ayuda a One-Hot

# 5. Preprocesamiento con ColumnTransformer
# Para el tratamiento, solemos utilizar: Media, mediana y/o moda
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()) # denso; aqui si con media
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

from sklearn.pipeline import Pipeline
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"

)
# 6. Split Train/Test estratificado
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

#Ajustar transformadores en train y transformar ambos
X_train = preprocess.fit_transform(X_train_df)
X_test = preprocess.transform(X_test_df)

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

print("Input dims: ", X_train.shape[1])

# 7. Definir y Compilar el modelo
def build_model(input dim: int) -> tf.keras.Model:
  model = models.Sequential([
      layers.Input(shape=(input_dim,)),
      layers.Dense(32, activation="relu"),
      layers.Dropout(0.15), #Apaga o desactiva ese porcentaje de neuronas. Es para evitar sobre ajustes. La IA debe aprender, no memorizar.
      layers.Dense(16, activation="relu"),
      layers.Dropout(0.15),
      layers.Dense(1, activation="sigmoid")
  ])
  model.compile(
      optimizer="adam",
      loss="binary_crossentropy", #perdida binaria
      metrics=["accuracy", tf.keras.metrics.AUC(name="auc")] #accuracy del modelo
  )
  return model

model = build_model(X_train.shape[1])
model.summary()

# 8. Callbacks (buenas practicas)
# Guarda los buenos resultados. Es para que pueda quedarse con el mejor dato para el modelo. No se quiere el ultimo, si no el mejor. Es por eso que se realiza callback, para guardar resultados

cbs = [
    callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=12, restore_best_weights=True),
    callbacks.ModelCheckpoint(filepath="titanic_best.keras", monitor="val_auc", mode="max", save_best_only=True), #Se realiza un checkpoint o guardado para el callback
    callbacks.ReduceLROnPlateau(monitor="val_auc", factor=0.5, patience=6)
]

# 9. Entrenamiento
hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=200,
    callbacks=cbs,
    verbose=1
)

# 10. Evaluacion en test
y_proba = model.predict(X_test).ravel()
y_pred = (y_proba >= 0.5).astype(int)

print("\nMatriz de confusion:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificacion:\n", classification_report(y_test, y_pred, digits=4))
print("\nAUC: ", roc_auc_score(y_test, y_proba))

# 11. Indiferencia robusta: funcion predict_one(dict de entrada)
def predict_one(sample: dict) -> float:
  """
  Recibe un diccionario 'crudo' con las llaves esperadas:
  Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
  (FamilySize e IsAlone se calculan internamente).
  Devuelve probabilidad de supervivencia.
  """

  #Convertir a DataFrame con columnas en orden esperado
  s = pd.DataFrame([sample])
  # Feature engineering consistente
  s["FamilySize"] = s["SibSp"].fillna(0) + s["Parch"].fillna(0) + 1
  s["IsAlone"] = (s["FamilySize"] == 1).astype(int)
  # Aplicar EXACTAMENTE el mismo preprocesamiento
  s_proc = preprocess.transform(s[X.columns]) #mismas columnas base
  s_proc = s_proc.astype("float32")
  #Predecir
  proba = model.predict(s_proc).item()
  return proba

  #Ejemplo de indiferencia
  sample = {
      "Pclass": 3, #1, 2 o3
      "Sex": "male",#male / female
      "Age": 25,
      "SibSp": 1,
      "Parch": 0,
      "Fare": 7.25,
      "Embarked": "S" #S, C o Q
  }
  proba = predict_one(sample)
  print(f"Probabilidad de supervivencia: {proba:.4f}")
  print("Sobrevive" if proba >= 0.5 else "No sobrevive")

  #

SyntaxError: invalid syntax (ipython-input-424201401.py, line 96)

In [ ]:
# Mas capas para más precisión o para qaue esté equilibrado. Generalmente es ensayo y error
